# Reproducibility results

This notebook compute the errors on each of the patches downloaded with the script `reproducibility.py`. Therefore, before running this notebook, it is expected that the following code has been run:

```
mkdir reproducibility_results
python reproducibility.py DownloadAll --basepath reproducibility_results --method percentile
```

In [2]:
from ee_ipl_uv import local_image
from sklearn.metrics import accuracy_score,confusion_matrix
from glob import glob
import numpy as np

def compute_confusion(raster_img):
    band_names = raster_img.bandNames()

    cloud_score = raster_img.memmap[...,band_names.index("cloudscore")]
    fixedmask = raster_img.memmap[...,band_names.index("fixedmask")]
    fixedmask = np.ma.masked_array(fixedmask,np.isnan(fixedmask))
    fixedmask = np.int16((fixedmask == 192) | (fixedmask == 255))
    fmask = raster_img.memmap[...,band_names.index("fmask")]
    fmask = np.ma.masked_array(fmask,np.isnan(fmask))
    fmask = np.int16(fmask == 4)

    cloud_score = np.ma.masked_array(cloud_score,np.isnan(cloud_score))
    
    mascara = np.ma.getmaskarray(fixedmask) | np.ma.getmaskarray(cloud_score) | np.ma.getmaskarray(fmask)
    cm_multi = confusion_matrix(np.ma.getdata(fixedmask)[~mascara],
                                np.ma.getdata(cloud_score)[~mascara],labels=[0,1])
    cm_fmask = confusion_matrix(np.ma.getdata(fixedmask)[~mascara],
                                np.ma.getdata(fmask)[~mascara],labels=[0,1])
    return cm_multi,cm_fmask


raster_img = local_image.LocalImage.Load("reproducibility_results/LC80150312014226LGN00_009_006_percentile")

cm1,cm2 = compute_confusion(raster_img)

assert np.sum(cm1) == np.sum(cm2), "Same number of pixels taken into account"


In [3]:
import os
from utils_ipl_uv import scores

files_read = glob("reproducibility_results/*_percentile")

cm_multi = np.zeros((2,2))
cm_fmask = np.zeros((2,2))

i = 0
for f in files_read:    
    if not os.path.exists(os.path.join(f,"raster.tif")):
        print("file %s does not exists"%f)
        continue
    i+=1
    raster_img = local_image.LocalImage.Load(f)
    cm1,cm2 = compute_confusion(raster_img)
    assert np.sum(cm1) == np.sum(cm2), "Same number of pixels taken into account"
    print("%d/%d %s %.4f %.4f"%(i,len(files_read),f,scores.accuracy_score_confusion(cm1),
                               scores.accuracy_score_confusion(cm2)))
    cm_multi+=cm1
    cm_fmask+=cm2
    
print("Processed %d files correctly"%i)    

1/1825 reproducibility_results/LC80980762014216LGN00_003_010_percentile 1.0000 1.0000
2/1825 reproducibility_results/LC80290372013257LGN00_011_012_percentile 0.9438 0.9090
file reproducibility_results/LC81360302014162LGN00_001_008_percentile does not exists
3/1825 reproducibility_results/LC81750732014035LGN00_010_004_percentile 0.9554 0.8834
4/1825 reproducibility_results/LC81180382014244LGN00_004_009_percentile 1.0000 1.0000
5/1825 reproducibility_results/LC81820302014180LGN00_005_012_percentile 1.0000 0.9998
6/1825 reproducibility_results/LC81590362014051LGN00_005_004_percentile 0.9954 1.0000
7/1825 reproducibility_results/LC81990402014267LGN00_012_010_percentile 0.9994 0.9919
8/1825 reproducibility_results/LC81020802014100LGN00_011_001_percentile 0.9416 0.9424
9/1825 reproducibility_results/LC80290372013257LGN00_010_012_percentile 0.8875 0.8470
10/1825 reproducibility_results/LC80410372013357LGN00_008_004_percentile 1.0000 1.0000
11/1825 reproducibility_results/LC81390292014135LGN00

90/1825 reproducibility_results/LC81590362014051LGN00_008_011_percentile 0.9925 0.9925
91/1825 reproducibility_results/LC81660432014020LGN00_010_004_percentile 1.0000 0.9990
92/1825 reproducibility_results/LC81490432014141LGN00_009_002_percentile 1.0000 1.0000
93/1825 reproducibility_results/LC81320352013243LGN00_010_008_percentile 0.9864 0.8013
94/1825 reproducibility_results/LC80640452014041LGN00_007_004_percentile 1.0000 1.0000
95/1825 reproducibility_results/LC80980762014216LGN00_013_011_percentile 0.9903 1.0000
96/1825 reproducibility_results/LC80290372013257LGN00_005_006_percentile 0.9342 0.9379
97/1825 reproducibility_results/LC80150312014226LGN00_013_009_percentile 0.9235 0.8410
98/1825 reproducibility_results/LC81660432014020LGN00_010_008_percentile 1.0000 1.0000
99/1825 reproducibility_results/LC80980712014024LGN00_007_012_percentile 0.9053 0.7335
100/1825 reproducibility_results/LC81590362014051LGN00_009_010_percentile 0.8719 0.5228
101/1825 reproducibility_results/LC8118038

178/1825 reproducibility_results/LC81590362014051LGN00_010_009_percentile 0.9815 0.7600
179/1825 reproducibility_results/LC81590362014051LGN00_012_007_percentile 0.9718 0.4413
180/1825 reproducibility_results/LC81570452014213LGN00_009_001_percentile 0.7224 0.9241
181/1825 reproducibility_results/LC80640452014041LGN00_013_005_percentile 1.0000 1.0000
182/1825 reproducibility_results/LC80290372013257LGN00_006_005_percentile 0.8563 0.8042
183/1825 reproducibility_results/LC80460282014171LGN00_006_009_percentile 0.9640 0.8132
184/1825 reproducibility_results/LC80980712014024LGN00_003_003_percentile 0.9327 0.7054
185/1825 reproducibility_results/LC80460282014171LGN00_014_009_percentile 0.9715 0.9457
186/1825 reproducibility_results/LC81020802014100LGN00_012_008_percentile 0.8283 0.8071
187/1825 reproducibility_results/LC80290372013257LGN00_002_012_percentile 0.9953 0.9536
188/1825 reproducibility_results/LC81750732014035LGN00_010_012_percentile 0.9277 0.8298
189/1825 reproducibility_results

267/1825 reproducibility_results/LC80980712014024LGN00_008_003_percentile 0.9677 0.9496
268/1825 reproducibility_results/LC81570452014213LGN00_010_003_percentile 1.0000 1.0000
269/1825 reproducibility_results/LC81360302014162LGN00_011_000_percentile 1.0000 1.0000
270/1825 reproducibility_results/LC81390292014135LGN00_004_014_percentile 0.9263 0.8097
271/1825 reproducibility_results/LC81620432014072LGN00_008_013_percentile 0.9954 0.9486
272/1825 reproducibility_results/LC81590362014051LGN00_007_007_percentile 0.9826 0.9588
273/1825 reproducibility_results/LC80290372013257LGN00_011_007_percentile 0.9931 0.9944
274/1825 reproducibility_results/LC80290372013257LGN00_009_003_percentile 0.9884 0.9774
275/1825 reproducibility_results/LC80980712014024LGN00_006_010_percentile 0.9655 0.7572
276/1825 reproducibility_results/LC81570452014213LGN00_004_004_percentile 0.9339 0.9361
277/1825 reproducibility_results/LC80980712014024LGN00_010_004_percentile 0.9637 0.8946
278/1825 reproducibility_results

358/1825 reproducibility_results/LC81990402014267LGN00_003_014_percentile 0.9890 0.8057
359/1825 reproducibility_results/LC81220312014208LGN00_013_007_percentile 0.9994 0.9920
360/1825 reproducibility_results/LC81820302014180LGN00_006_003_percentile 1.0000 1.0000
361/1825 reproducibility_results/LC80290372013257LGN00_003_004_percentile 0.9308 0.9204
362/1825 reproducibility_results/LC81490432014141LGN00_013_007_percentile 0.9999 1.0000
363/1825 reproducibility_results/LC81020802014100LGN00_007_010_percentile 0.9893 0.9893
364/1825 reproducibility_results/LC80290372013257LGN00_002_010_percentile 1.0000 1.0000
365/1825 reproducibility_results/LC81620432014072LGN00_006_008_percentile 0.8518 0.5420
366/1825 reproducibility_results/LC81990402014267LGN00_013_004_percentile 0.5035 0.8597
367/1825 reproducibility_results/LC81660432014020LGN00_011_009_percentile 1.0000 1.0000
368/1825 reproducibility_results/LC81750732014035LGN00_011_005_percentile 0.9549 0.9356
369/1825 reproducibility_results

...

1338/1825 reproducibility_results/LC80640452014041LGN00_002_003_percentile 1.0000 0.9999
file reproducibility_results/LC81590362014051LGN00_014_008_percentile does not exists
1339/1825 reproducibility_results/LC81750732014035LGN00_004_008_percentile 0.8639 0.6627
1340/1825 reproducibility_results/LC81490432014141LGN00_013_004_percentile 0.6523 1.0000
1341/1825 reproducibility_results/LC81660432014020LGN00_003_014_percentile 1.0000 1.0000
1342/1825 reproducibility_results/LC81590362014051LGN00_009_008_percentile 0.9821 0.9744
1343/1825 reproducibility_results/LC81590362014051LGN00_008_001_percentile 0.9573 0.8294
1344/1825 reproducibility_results/LC81990402014267LGN00_005_011_percentile 0.9432 0.9459
file reproducibility_results/LC81180382014244LGN00_001_008_percentile does not exists
1345/1825 reproducibility_results/LC81590362014051LGN00_009_004_percentile 1.0000 1.0000
1346/1825 reproducibility_results/LC81590362014051LGN00_009_002_percentile 0.9991 0.9967
1347/1825 reproducibility_r

1423/1825 reproducibility_results/LC81750732014035LGN00_010_007_percentile 0.9710 0.9711
1424/1825 reproducibility_results/LC81750732014035LGN00_002_006_percentile 0.9141 0.8225
1425/1825 reproducibility_results/LC80290372013257LGN00_005_004_percentile 0.9126 0.8924
1426/1825 reproducibility_results/LC81360302014162LGN00_006_008_percentile 0.9270 0.9270
1427/1825 reproducibility_results/LC80980712014024LGN00_003_006_percentile 0.9513 0.3048
1428/1825 reproducibility_results/LC81570452014213LGN00_005_008_percentile 0.9986 0.9986
1429/1825 reproducibility_results/LC80290372013257LGN00_012_007_percentile 0.9757 0.9624
1430/1825 reproducibility_results/LC81360302014162LGN00_011_006_percentile 0.7783 0.7783
1431/1825 reproducibility_results/LC80460282014171LGN00_002_006_percentile 0.9365 0.8530
1432/1825 reproducibility_results/LC81180382014244LGN00_008_011_percentile 1.0000 1.0000
1433/1825 reproducibility_results/LC81490432014141LGN00_009_008_percentile 1.0000 1.0000
1434/1825 reproducibi

1513/1825 reproducibility_results/LC80980712014024LGN00_008_012_percentile 0.9138 0.7933
1514/1825 reproducibility_results/LC81360302014162LGN00_013_011_percentile 0.2968 0.2968
1515/1825 reproducibility_results/LC80410372013357LGN00_004_007_percentile 1.0000 1.0000
1516/1825 reproducibility_results/LC81590362014051LGN00_011_010_percentile 0.9052 0.9273
1517/1825 reproducibility_results/LC81990402014267LGN00_010_002_percentile 0.9978 0.9809
1518/1825 reproducibility_results/LC81020802014100LGN00_012_004_percentile 0.9158 0.8983
1519/1825 reproducibility_results/LC80290372013257LGN00_011_002_percentile 0.9245 0.9045
1520/1825 reproducibility_results/LC80460282014171LGN00_002_008_percentile 0.9388 0.8055
1521/1825 reproducibility_results/LC80980712014024LGN00_003_010_percentile 0.9123 0.6792
1522/1825 reproducibility_results/LC81570452014213LGN00_009_010_percentile 0.9594 0.2523
1523/1825 reproducibility_results/LC81490432014141LGN00_006_005_percentile 0.9985 1.0000
1524/1825 reproducibi

1604/1825 reproducibility_results/LC81020802014100LGN00_011_011_percentile 0.9999 0.9999
1605/1825 reproducibility_results/LC80980712014024LGN00_008_011_percentile 0.9321 0.6048
1606/1825 reproducibility_results/LC80980712014024LGN00_004_012_percentile 0.9510 0.5757
1607/1825 reproducibility_results/LC81990402014267LGN00_013_007_percentile 0.7373 0.5715
1608/1825 reproducibility_results/LC80150312014226LGN00_004_012_percentile 0.9017 0.8602
1609/1825 reproducibility_results/LC81660432014020LGN00_002_005_percentile 1.0000 1.0000
file reproducibility_results/LC81660432014020LGN00_007_003_percentile does not exists
1610/1825 reproducibility_results/LC81750732014035LGN00_008_011_percentile 0.8927 0.8650
file reproducibility_results/LC80290372013257LGN00_000_004_percentile does not exists
1611/1825 reproducibility_results/LC81660432014020LGN00_007_004_percentile 1.0000 1.0000
1612/1825 reproducibility_results/LC81590362014051LGN00_010_004_percentile 0.9932 0.9670
1613/1825 reproducibility_r

1693/1825 reproducibility_results/LC80410372013357LGN00_006_005_percentile 1.0000 1.0000
1694/1825 reproducibility_results/LC81990402014267LGN00_007_001_percentile 0.9365 0.9388
1695/1825 reproducibility_results/LC81020802014100LGN00_005_013_percentile 1.0000 1.0000
1696/1825 reproducibility_results/LC81220312014208LGN00_010_001_percentile 1.0000 0.9989
1697/1825 reproducibility_results/LC81990402014267LGN00_005_003_percentile 0.9663 0.9300
1698/1825 reproducibility_results/LC80980762014216LGN00_002_012_percentile 1.0000 1.0000
1699/1825 reproducibility_results/LC80980762014216LGN00_007_013_percentile 1.0000 1.0000
file reproducibility_results/LC81750732014035LGN00_001_008_percentile does not exists
1700/1825 reproducibility_results/LC81220312014208LGN00_010_009_percentile 1.0000 1.0000
1701/1825 reproducibility_results/LC81390292014135LGN00_011_012_percentile 0.9377 0.8647
1702/1825 reproducibility_results/LC81570452014213LGN00_003_002_percentile 0.8092 0.4166
1703/1825 reproducibilit

In [7]:
scores.accuracy_score_confusion(cm_multi),scores.accuracy_score_confusion(cm_fmask)

(0.941776266611413, 0.8825396553208895)

In [8]:
scores.commission_error(cm_multi),scores.commission_error(cm_fmask)

(0.056969383792937076, 0.16503740377024353)

In [9]:
scores.omission_error(cm_multi),scores.omission_error(cm_fmask)

(0.06065124551785184, 0.025385624506193367)